In [1]:
import random

In [2]:
def read_city_files(name_file, distance_file):
    """
    Description:
    This function takes the file location for the city names file and
    the file location for the distances file and outputs two lists for
    using in other functions.

    Parameters:
    name_file (string): file containing the names of n cities
    distance_file (string): file containing an nxn matrix of distances between cities

    Returns:
    list of names and an nxn matrix of distances between cities
    """

    with open(name_file) as file:

        city_names = []
        for line in file:
            city_names.append(line.strip())

    with open(distance_file) as file:

        distances = []
        for line in file:
            line = line.strip()
            split_line = line.split()
            for i in range(len(split_line)):
                split_line[i] = float(split_line[i].strip())

            distances.append(split_line)

        return(city_names, distances)

In [3]:
def calc_distance(path_list, distances):
    """
    This function takes the path taken and distances between each city and
    calculates the total distance between the cities

    Parameters:
    path_list (list[int]): a list of cities visited in order
    distances (list[list[float]]): an nxn matrix where the distance of city i and city j
    is at (i,j) and (j,i)

    Returns:
    float: the total distance of the input path:
    """

    total_distance = 0
    # Sums distance of ith city visited to i+1th city visited except the last city to the original city

    for i in range(len(path_list) - 1):
        current_city = path_list[i]
        next_city = path_list[i + 1]
        total_distance += distances[current_city][next_city]

    # adds the distance of the last city to the original city
    total_distance += distances[path_list[-1]][path_list[0]]

    return total_distance

# Brute Force via Recursion
### Peter

# Pseudo-Code
## *Backtracking*
### *tsp_backtrack*
    takes in list of n cities, NxN array of distances
    base_city
    for city 1 from 1 to number citites

        for city 2 from city 1 to number of cities

        tsp_recursion takes in [city_1, base_city, city_2], list of all cities

### *tsp_recursion*
        takes in current cities in path and a list of all cities
        remove all cities currently in the path from the list of all cities
        if there are no cities to add to the path
            return the distance
        else:
            min_distance = -1
            min_path
            for all cities remaining

                append one at a time to the path and call tsp_recursion(appended_path, list of all cities)
                current_path, current_distance = tsp_recursion(appended_path, list of all cities)
                if min_distance = -1 or min_distance > current_distance
                    min_distance = current_distance
                    min_path = current_path

    return(min_path, min_distance)

In [4]:
def tsp_recursion(seed_path, remaining_cities, distances):
    """
    Description:
    A recursive function for tsp_backtrack, it takes in a list of cities visited in order and
    a list of cities yet to be visited. If there are no more remaining cities, it calculates the
    distance of the path and returns that. Otherwise, for each remaining city it appends the remaining
    city to the seed_path and removes the city from the remaining cities and feeds the new_seed_path,
    new_remaining_cities and distances to a tsp_recursion (new_seed_path will only have one additional city
    at a time). The function saves the shortest distance and path and returns that.

    Parameters:
    seed_path (list[int]): list of cities currently visited in order
    remaining_cities (list[int]): list of cities not yet visited in order
    distances (list[list[float]]): an nxn matrix where the distance of city i and city j)

    Returns:
    min_path (list[int]): a list of cities visited in order that is the shortest path found
    min_distance (float): distance of the shortest path
    """

    if len(remaining_cities) == 0:
        distance = calc_distance(seed_path, distances)
        return seed_path, distance
    else:
        min_path = None
        min_distance = -1

        for i in remaining_cities:

            # for each city makes a list of remaining cities without
            # a singular city
            filtered_list = remaining_cities.copy()
            filtered_list.remove(i)

            # adds that city to a new seed path
            partial_path = seed_path.copy()
            partial_path.append(i)
            path, dist = tsp_recursion(partial_path, filtered_list, distances)

            # if the distance of the seed path is shorter
            # than each other possible new seed path
            # save it as the min_path and min_distance
            if (dist < min_distance) or (min_distance == -1):
                min_path = path
                min_distance = dist

        return min_path, min_distance


In [5]:
def tsp_backtrack(name_file, distance_file):
    """
    Description:
    The seed function for tsp_recursive. It creates all potential seed "directions" as to allow for recursion
    to check each path without checking the same path more than once. It calls tsp_recursion for each seed, saving
    the one with the shortest distance and path and returns those.

    Parameters:
    name_file (string): file containing the names of n cities
    distance_file (string): file containing an nxn matrix of distances between cities

    Returns:
    named_path (list[int]): a list of city names visited in order that is the shortest path found
    min_distance (float): distance of the shortest path
    """
    names, distances = read_city_files(name_file, distance_file)

    list_of_cities = list(range(len(names)))

    base_city = list_of_cities[0]

    min_path, min_dist = None, None

    # creates each unique seed path because city_2 starts as the city directly after city_1
    for city_1 in range(1, len(list_of_cities)):
        for city_2 in range(city_1+1, len(list_of_cities)):
            remaining_cities_list = list_of_cities.copy()

            remaining_cities_list.remove(base_city)
            remaining_cities_list.remove(list_of_cities[city_1])
            remaining_cities_list.remove(list_of_cities[city_2])

            seed_path = [city_1, base_city, city_2]

            path, dist = tsp_recursion(seed_path, remaining_cities_list, distances)

            if min_dist == None or dist < min_dist:
                min_dist = dist
                min_path = path

    named_path = []
    for i in min_path:
        named_path.append(names[i])

    return named_path, min_dist

In [6]:
min_path, min_dist = tsp_backtrack('Data/seven_cities_names.txt', 'Data/seven_cities_dist.txt')
print(f"The shortest path found is {min_path}\n"
      f"The shortest distance found is {min_dist}")

The shortest path found is ['Epsilon', 'Alpha', 'Eta', 'Beta', 'Zeta', 'Delta', 'Gamma']
The shortest distance found is 106.4


# *Evolutionary Algorithm*
### Joey

# *Pseudocode For Evolutions*

    generate a list of the cities in a random order
    select a random city to move to that hasn't been visited
    if all cities have been visited select two cities and
    swap where they are in the visit order

    calculate distance (fitness metric)

    if distance is less than the current min save it as the current min

    if none of the offspring improve on the parent

    save parent as a local min

    mutate 5 times

    repeat mutations until new min is found

    once 3 local min are found return the most fit

In [4]:
def calculate(path, distance):
    '''
    description:
    setting total distance equal to 0 to start and then for each index in the range of the path set the current city equal to the index of the loop.
    if the index is at the end of the path, the next city is the first element in the path. Add the distances between the current city and the next
    city to total distance


    parameter:
    path which is the inputted path
    distance which is in the distance_file and is the distances of the current path

    return:
    total distance of the current city and next cities, i.e. the total distance of a given path
    '''
    total_dist = 0

    for i in range(len(path)):
        current_city = path[i]
        if i == len(path) - 1:
            next_city = path[0]
        else:
            next_city = path[i + 1]

        total_dist += distance[current_city][next_city]

    return total_dist

In [5]:
def tsp_mutate(path):
    '''
    description:
    creates a sample that takes in the input path and randomly chooses an index, it then removes that index and appends it to the end

    parameters:
    path which eventually will be the given path in the evolution function

    return:
    returns a path that removes a random element and appends it to the end
    '''
    sample = random.sample(path, 1)[0]

    path.remove(sample)
    path.append(sample)

    return path

In [6]:
def tsp_evolution(name_file, distance_file):
    '''
    Description:
    Takes in 2 inputs, name_file and distance_file and returns the minimum distances in a certain path. The function sets a local minimum path
    and minimum distance and creates an empty list that will be appended with the eventual minimum path and distance. The function goes through
    a given number of mutations in order to find the minimum path based on the mutate and calculate functions. If one of the mutated distances
    are smaller than the current local minimum, a new local minimum is created. After all mutations are completed, the final local minimum is
    appended to the empty list.

    Parameters:
    name_file which is a file of city names
    distance_file which is a file of distances between each city

    Returns:
    Returns a path with the minimum found distance
    '''

    # set names and distances equal to read_city_file
    # create and initialize beginning path
    names, distances = read_city_files(name_file, distance_file)

    path = list(range(len(names)))
    random.shuffle(path)

    # set local minimums
    local_min_path = path
    local_min_dist = calculate(path, distances)

    local_mins = []

    while len(local_mins) < 3:

        mutated = False

        # set the local minimum path equal to parent path
        parent_path = local_min_path

        for i in range(100):

            mutation = tsp_mutate(parent_path)  # randomize the parent path
            mutation_dist = calculate(mutation,
                                      distances)  # calculate the mutation distance for the given mutation path

            if mutation_dist < local_min_dist:
                mutated = True
                local_min_dist = mutation_dist
                local_min_path = mutation

        if mutated == False:
            local_mins.append((local_min_path, local_min_dist))

            for i in range(5):
                local_min_path = tsp_mutate(local_min_path)

    # find overall best minimum
    minimum = local_mins[0]
    for min_path in local_mins:
        if min_path[1] < minimum[1]:
            minimum = min_path

            # convert city indices to names
    named_path = []
    for i in minimum[0]:
        named_path.append(names[i])
    minimum = (named_path, minimum[1])
    return minimum  # returning best path and minimum

In [7]:
tsp_evolution("Data/seven_cities_names.txt", "Data/seven_cities_dist.txt")

(['Beta', 'Alpha', 'Gamma', 'Delta', 'Zeta', 'Eta', 'Epsilon'], 114.3)

In [8]:
tsp_evolution("Data/thirty_cities_names.txt", "Data/thirty_cities_dist.txt")

(['Mexico City',
  'Rio de Janeiro',
  'London',
  'Guam',
  'Bombay',
  'Azores',
  'New York',
  'Buenos Aires',
  'Manila',
  'Montreal',
  'Sydney',
  'New Orleans',
  'Seattle',
  'Melbourne',
  'Shanghai',
  'Rome',
  'San Francisco',
  'Honolulu',
  'Panama City',
  'Istanbul',
  'Berlin',
  'Paris',
  'Santiago',
  'Juneau',
  'Moscow',
  'Capetown',
  'Tokyo',
  'Cairo',
  'Chicago',
  'Baghdad'],
 1385.0)